In [99]:
'''
Title: Batter Performance Table
Author: Sam LaFell
Date: 11/5/2022
Purpose: I got a little carried away on my last notebook. Time to optimize some code.

Granularity: xBA by pitch for each batter. By pitcher handiness.
'''

# Import Libraries
import pandas as pd
import pybaseball as pb
import os
import numpy as np
import re

pd.set_option('display.max_columns', 500)

In [97]:

# Statcast
from pybaseball import statcast
data = statcast('2018-03-30', '2018-03-30', team='ATL')

This is a large query, it may take a moment to complete


100%|██████████| 1/1 [00:03<00:00,  3.11s/it]


In [42]:
# The batters before this game
season_estimated_ba = data.groupby('batter').agg({'estimated_ba_using_speedangle':'mean'}).reset_index()

In [148]:
# If Game Date is after 5/1, do this
game_day = '2018-05-13'
from datetime import timedelta, datetime
first_day_range = (datetime.strptime(game_day, '%Y-%m-%d') - timedelta(days = 21)).strftime('%Y-%m-%d')
last_day_range = (datetime.strptime(game_day, '%Y-%m-%d') - timedelta(days = 1)).strftime('%Y-%m-%d')

# Who is playing?
team = 'ATL'

# Pull down Data
data = statcast(first_day_range, last_day_range, team=team)

# The batters before this game
season_estimated_ba = data.groupby('batter').agg({'estimated_ba_using_speedangle':'mean'}).reset_index()

# PAs before this game
atbats = data.groupby(['game_date', 'batter', 'at_bat_number']).agg({'batter':'count'})
atbats.columns = ['batter_count']
atbats = atbats.reset_index()
season_atbats = atbats.drop(columns=['at_bat_number'], axis=1).groupby('batter').sum(numeric_only=True).reset_index()

# Batters PAs and xBA to date
pas_xba = season_atbats.merge(season_estimated_ba, how='inner', on='batter').rename(columns={'batter_count':'PA', 'estimated_ba_using_speedangle':'xBA'})
pas_xba.sort_values('PA', ascending=False)

This is a large query, it may take a moment to complete


100%|██████████| 21/21 [00:06<00:00,  3.29it/s]


,batter,PA,xBA
16,458015,77,0.3605
45,571697,76,0.322167
71,606299,67,0.278471
58,594807,67,0.274
46,571740,63,0.279
...,...,...,...
90,623149,3,<NA>
75,607237,3,<NA>
64,601713,3,<NA>
5,450306,2,<NA>


In [153]:
# Reverse Player Lookup
from pybaseball import playerid_reverse_lookup
# find the names of the players in player_ids, along with their ids from other data sources
batters_lookup = playerid_reverse_lookup(pas_xba.batter.values, key_type='mlbam')
batters_lookup.loc[batters_lookup['key_mlbam'] == 458015] # he does NOT play for ATL

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
94,votto,joey,458015,vottj001,vottojo01,4314,2007.0,2022.0


In [151]:
# Find the name team and season
from pybaseball import batting_stats
game_year = datetime.strptime(game_day, '%Y-%m-%d').year
# get all of this season's batting data so far
team_data = batting_stats(game_year, qual=5)

# Names
team_data['name_first'] = team_data.Name.str.split(" ").str[0].str.lower()
team_data['name_last'] = team_data.Name.str.split(" ").str[1].str.lower()
enhanced_player_lookup_pt1 = team_data[['IDfg', 'Season', 'Name', 'Team', 'Age', 'name_first', 'name_last']]

# Join
enhanced_player_lookup = batters_lookup.merge(enhanced_player_lookup_pt1, how='inner', on=['name_first', 'name_last'])


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,IDfg,Season,Name,Team,Age


In [152]:
# Only Atlanta
enhanced_player_lookup.Team.value_counts()

PHI      15
CIN      14
- - -    14
NYM      14
SFG      13
MIA      11
TBR       7
Name: Team, dtype: int64

In [103]:
description = [*data.des][0]
last = description.split(' ')[1]
first = description.split(' ')[0]
players = pb.playerid_lookup(last=last)
batter = players.loc[players['name_first'] == first.lower()]

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
18,castro,starlin,516770,casts001,castrst01,4579,2010.0,2021.0


In [139]:
# Find the name team and season
from pybaseball import batting_stats
game_year = datetime.strptime(game_day, '%Y-%m-%d').year
# get all of this season's batting data so far
team_data = batting_stats(game_year, qual=5)

# Names
team_data['name_first'] = team_data.Name.str.split(" ").str[0].str.lower()
team_data['name_last'] = team_data.Name.str.split(" ").str[1].str.lower()
enhanced_player_lookup_pt1 = team_data[['IDfg', 'Season', 'Name', 'Team', 'Age', 'name_first', 'name_last']]

# Join
enhanced_player_lookup = batter.merge(enhanced_player_lookup_pt1, how='inner', on=['name_first', 'name_last'])

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,IDfg,Season,Name,Team,Age
0,castro,starlin,516770,casts001,castrst01,4579,2010.0,2021.0,4579,2018,Starlin Castro,MIA,28


In [114]:
team_data.loc[team_data['Name'] == first + ' ' + last]

,Name,Team,Season
246,Starlin Castro,MIA,2018


In [132]:
team_data['First_Name'] = team_data.Name.str.split(" ").str[0]
team_data['Last_Name'] = team_data.Name.str.split(" ").str[1]

In [ ]:
team_data['Last_Name'] = team_data.Name.str.split(" ")[1]